In [4]:
from llama_index.llms.dashscope import DashScope
import os, sys
sys.path.append('..')
from chatbot import rag

api_key = os.getenv("DASHSCOPE_API_KEY")

index = rag.create_index('../chatbot/docs')
query_engine = index.as_query_engine(
  streaming=True,
  llm=DashScope(
    model_name="qwen-plus",
    api_key=api_key
  ),
  similarity_top_k=3
)

response = rag.ask("张伟是哪个部门的", query_engine=query_engine, debug=True)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/miniconda3/envs/ACPEnv/lib/python3.13/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.




检索召回的文档切片如下：

[1] Node ID: dbb75b87-6bb5-4b47-b002-b836787ed87a
Text: com|进行公司网络及硬件设备的配置与维护,监控系统运行状态,及时处理技术问题与故障,提供技术支持及工具使用培训。| | IT部
|谢宇|马云|H803|037|IT支撑|IT专员|13800000037|xieyu@educompany.com|支持公司软件系统的开发
与更新,参与IT项目的计划与实施,撰写技术文档与使用说明,确保信息技术的安全性与有效性。| |
绩|朱|李|I901|04|人|绩|1380000004|zhun@educompany.com|负| | 效管理部|南|飞||0|力资源|
效专员|0||责制定绩效考核体系,组织绩效评估的实施与反馈,撰写评估报告,分析绩效数据以提出优化建议,提供决策支持。| |
绩效管理部|韩杉|李飞|1902|041|人力资源|绩效专员|13...
Score:  0.291


[2] Node ID: 5a7744a2-2ce0-459c-bda2-6ce8060c707c
Text: 内容公司各部门职责与关键角色联系方式 各部门关键角色联系人 |
部门|员工姓名|员工主管|工位|工号|岗位|职位|电话|邮箱|工作职责| |
---|---|---|---|---|---|---|---|---|---| | 教研部|张伟|李琳|A101|001|内容设计|教研专
员|13800000001|zhangwei@educompany.com|负责教育课程的研究与开发,分析教学效果,整理教案,协助课程优化,
以及参与教育项目的评估和反馈。| | 教研部|王芳|李琳|A102|002|内容设计|教研专员|13800000002|wangfang@e
ducompany.com|负责制定学科教学方案,策划教学活动,编写教案,收集学生反馈,参与课程改进会议,提供专业意见。| | 教研...
Score:  0.269


[3] Node ID: 2a436664-7ec9-430b-abe8-0fb7d82598ef
Text: com|主持教材的市场调研,分析不不同阶段学生的学习需求,撰写教材使用报告,积极参与教材的试用和反馈收集,促进优化迭代。| | 
评估部|邓琪|张晓|D

In [2]:
!pip3 install -U llama-index-postprocessor-dashscope-rerank-custom

In [11]:
from llama_index.postprocessor.dashscope_rerank import DashScopeRerank
from llama_index.core.postprocessor import SimilarityPostprocessor
import sys
sys.path.append('..')
from chatbot import rag

query_engine = index.as_query_engine(
  llm=DashScope(
    model_name="qwen-plus",
    api_key=api_key
  ),
  # 先设置一个较大的召回切片数量
  similarity_top_k=10,
  streaming=True,
  node_postprocessors=[
    # 在rerank 模型中选择你最终想召回的切片个数，重排模型选择通义实验室的gte-rerank模型
    DashScopeRerank(top_n=3, model="gte-rerank"),
    # 设置一个相似度阈值，低于该阈值的切片会被过滤掉
    SimilarityPostprocessor(similarity_cutoff=0.3)
  ]
)

response = rag.ask("张伟是哪个部门的", query_engine=query_engine, debug=True)



检索召回的文档切片如下：

[1] Node ID: 5a7744a2-2ce0-459c-bda2-6ce8060c707c
Text: 内容公司各部门职责与关键角色联系方式 各部门关键角色联系人 |
部门|员工姓名|员工主管|工位|工号|岗位|职位|电话|邮箱|工作职责| |
---|---|---|---|---|---|---|---|---|---| | 教研部|张伟|李琳|A101|001|内容设计|教研专
员|13800000001|zhangwei@educompany.com|负责教育课程的研究与开发,分析教学效果,整理教案,协助课程优化,
以及参与教育项目的评估和反馈。| | 教研部|王芳|李琳|A102|002|内容设计|教研专员|13800000002|wangfang@e
ducompany.com|负责制定学科教学方案,策划教学活动,编写教案,收集学生反馈,参与课程改进会议,提供专业意见。| | 教研...
Score:  0.583


[2] Node ID: 2a436664-7ec9-430b-abe8-0fb7d82598ef
Text: com|主持教材的市场调研,分析不不同阶段学生的学习需求,撰写教材使用报告,积极参与教材的试用和反馈收集,促进优化迭代。| | 
评估部|邓琪|张晓|D405|021|内容质检|评估专员|13800000021|dengqi@educompany.com|协助评估项目的
相关文档整理,参与评估结果的反馈会议,提供数据支持,监督评估实施过程,确保评估的全面性与公正性。| | 市场部|郑刚|李明|E504|025
|市场昔营销|市场推专员|13800000025|zhenggang@educompany.com|开展市场竞品分析,收集行业动态信息,提供
市场预测,协助制定销售目标和策略,定期撰写市场研究报告。| |
人FR您源部|胡楠|马婷|F603|028|人力资源|人力资源专员|1380000...
Score:  0.563


[3] Node ID: dbb75b87-6bb5-4b47-b002-b836787ed87a
Text: com|进行公司网络及硬件设备的配置与维护,监控系统运行状态,及时处理技术问题与故障,提供技术支持及工具使用培训。| | IT部
|谢宇|马云|H80